# Clustering *League of Legends* Player Decisions by Game State
*League of Legends* is a online game divided into various matches in which teams of players compete against each other to destroy the others bases. The central gameplay loop revolves around completing various objectives in order to acquire gold which players exchange for items that make themselves more powerful. The items purchased by players represent one of the major choices available, as different items grant different powers to players. The purpose of this analysis is to analyze the choices made by those players to see if people actually care enough to vary their purchases relative to the game state, as decided by kill/death ratio as a proxy variable for individual success. Clustering will be done via K-means, and the primary question of this analysis will be identifying if the clusters of item choices correspond to game-states. 

In [1]:
import pandas as pd
from sklearn.cluster import KMeans

/home/ryancoops/miniconda3/envs/dsci235p/lib/python3.13/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


First, we will import our csv from the first notebook, Data_Collection.ipynb. This step has been separated out to remedy the issues with reading Kaggle data from cache, as that's an absolute path. The data includes the champion (the player's character), their role on the team, gameID data, and victory status. After that includes the seven item choices, signified by their four digit item IDs, and one-hot encodings for all the item choices.

In [5]:
items = pd.read_csv("data/encodedLeagueData.csv")
pd.DataFrame.head(items)

,championName,lane,GameID,win,item0,item1,item2,item3,item4,item5,...,7014,7015,7016,7017,7018,7019,7020,7021,7022,8001
0,Trundle,JUNGLE,4035079604,True,3075,2031,3044,3047,6632,1037,...,0,0,0,0,0,0,0,0,0,0
1,Sylas,MIDDLE,4035079604,True,4629,3157,6656,3020,1082,1058,...,0,0,0,0,0,0,0,0,0,0
2,Viego,JUNGLE,4035079604,False,3047,2031,6632,3153,1031,0,...,0,0,0,0,0,0,0,0,0,0
3,Yasuo,MIDDLE,4035079604,False,6333,3006,6673,3031,3072,0,...,0,0,0,0,0,0,0,0,0,0
4,Caitlyn,BOTTOM,4035079604,False,1055,6672,3006,3031,0,0,...,0,0,0,0,0,0,0,0,0,0
